<h1>Exploiting Genetic Algorithms for Optimisation </h1>
<br>

<h2> The problem</h2>
We need to make cylindrical containers with fixed volume 30 cubic cm so that, given the diameter you can figure out its height, minimizing the building material. 
This mathematically simple minimization problem could be solved with a genetic algorithm driven by the diameter (or radius) that must always be positive. The rest of the problem definition is up to you. 

In [80]:
# Install dependencies
import math
import numpy as np
import time
from tqdm import tqdm

## Defining the Problem

We need to optimise the building material of Cilinders of Volume 30
$$ 
\begin{equation}
    Volume = \pi r^2 h = 30
\end{equation}
$$

In [81]:

#1. Initial Hyperparameters
np.random.seed(2023)

#2. Define height given diameter
def height (d):
    r = d/2
    pi = math.pi
    #volume = pi * (r**2) * h
    h = 30/(pi*(r**2))
    return h

#3. Define Fitness
def fitness (r):
    #r=d/2
    return 1/(2 * (math.pi) * (r**2) + (60/r))
    
#3. Define selection methods
def ranked_fitness (current_generation, max_radius, n):
    ranked_solutions = []
    for el in current_generation:
        ranked_solutions.append((fitness(decode_r(el, max_radius, n)),decode_r(el, max_radius, n)))
        #added a second element to keep track of solutions and not only fitness
    ranked_solutions.sort()
    ranked_solutions.reverse()
    return ranked_solutions

#4. Define selection methods
def random_choice (generation, max_radius, n):
    choice = np.random.randint(0,len(generation)-1)
    return generation[choice]

def ranked_choice (generation, max_radius, n):
    #based on https://stackoverflow.com/questions/20290831/how-to-perform-rank-based-selection-in-a-genetic-algorithm
    gen_fit = []
    probabilities = []
    amax = 1.1
    amin = 2 - amax
    m = len(generation)
    
    for i in range(len(generation)-1):
        gen_fit.append((fitness(decode_r(generation[i], max_radius, n)),generation[i]))
        p = (amax - (amax-amin)*(i/(m-1)))*(1/m)
        probabilities.append(p)
    #make sure probs sum to 1    
    sum_probs = sum(probabilities)
    probabilities[0] = probabilities[0] + (1 - sum_probs)
    
    choice= gen_fit[np.random.choice(np.arange(len(gen_fit)), p=probabilities)]
    return choice[1]

def wheel_selection (generation, max_radius, n):
    #create list with radius and its fitness
    gen_fit=[]
    for el in generation:
        gen_fit.append((fitness(decode_r(el, max_radius, n)),el))           
    # computes the total the generation fitness
    generation_fitness = sum([el[0] for el in gen_fit])
    # computes the probability for each radius
    probabilities = [el[0]/generation_fitness for el in gen_fit]
    # selects one chromosome based on the computed probabilities
    choice= gen_fit[np.random.choice(np.arange(len(gen_fit)), p=probabilities)]
    return choice[1]


#5. Define Reproduction methods    
def crossover(parentA, parentB):
    cross_point = np.random.randint(len(parentA))
    child = parentA[:cross_point]+parentB[cross_point:]
    return child


#6. Reproduction Behaviour
def reproduction (current_gen, max_radius, n, sel_function):
    new_generation = []
    for i in range(len(current_gen)):
        parentA = sel_function(current_gen, max_radius, n)
        parentB = sel_function(current_gen, max_radius, n)
        child = crossover(parentA, parentB)
        if '1' in child:
            new_generation.append(child)
        else:
            new_generation.append(parentA)
    return new_generation

        
#Function to decode binary radius
def decode_r(r, max_radius, n):
    rad=list(map(int, str(r)))
    dec_r=(max_radius/(2**n-1))*sum(rad[i]*2**i for i in range(len(rad)))
    return dec_r

#Generation zero binary
def first_gen(pop_size,n):
    gen_zero=[]
    for i in range(pop_size):
        x = []
        while 1 not in x:
            x = [np.random.choice([0, 1]) for _ in range(n)]   
        el = ''.join(map(str, x))
        gen_zero.append(el)

    return gen_zero

In [82]:
#5. New generation iterations
def digi_evolve(pop_size, max_generations, max_radius, n, sel_function):
    #create temporary vars
    fittest = []
    curr_generation = first_gen(pop_size, n) #initialising first gen
    #GA begins
    print('Evolutionary period begins')
    for i in tqdm(range(max_generations)):
        #if i % 50 == 0:
        #    print(f'Generation {i} is going into the world')
        #    print(f'Population size {len(curr_generation)}')
        ranked_fit = ranked_fitness(curr_generation, max_radius, n)
        fittest.append(ranked_fit[0])
        #reproduction
        curr_generation = reproduction(curr_generation, max_radius, n, sel_function)
    print('Evolutionary period ended')
    return fittest    


In [83]:
#main
pop_size = 100
max_generations = 1000
n=8 #step:0.1176 if we work in range [0,30]
max_radius=30
selection_methods = [[random_choice, 'Random Selection'], [ranked_choice, 'Ranked Selection'],[wheel_selection, 'Wheel Selection']]
best_overall = []

start_total = time.time()
start_local = time.time()

for selectionMethod in selection_methods:
    print(f'Selection method: {selectionMethod[1]}')
    best_of_each_gen = digi_evolve(pop_size, max_generations, max_radius, n, selectionMethod[0])
    print('The best radius is:', max(best_of_each_gen)[1])
    best_overall.append([max(best_of_each_gen), selectionMethod[1]])
    end_local = time.time()
    print('Time in seconds to complete iteration of', selectionMethod[1] ,end_local - start_local, '\n')
    start_local = time.time()


end_total = time.time()
print('Time in seconds to complete entire cycle:',end_total - start_total, '\n')

print('The best radius is:', max(best_overall))

#do we want to add mutations too?


Selection method: Random Selection
Evolutionary period begins


100%|██████████| 1000/1000 [00:01<00:00, 626.99it/s]


Evolutionary period ended
The best radius is: 1.6470588235294117
Time in seconds to complete iteration Random Selection 1.60542893409729 

Selection method: Ranked Selection
Evolutionary period begins


100%|██████████| 1000/1000 [01:45<00:00,  9.47it/s]


Evolutionary period ended
The best radius is: 1.6470588235294117
Time in seconds to complete iteration Ranked Selection 105.56290411949158 

Selection method: Wheel Selection
Evolutionary period begins


100%|██████████| 1000/1000 [01:43<00:00,  9.70it/s]

Evolutionary period ended
The best radius is: 1.6470588235294117
Time in seconds to complete iteration Wheel Selection 103.095134973526 

Time in seconds to complete entire cycle: 210.26392078399658 

The best radius is: [(0.01870081198259601, 1.6470588235294117), 'Wheel Selection']


In [84]:
#Compare it with random choice

start = time.time()

rand_best_of_each_gen = digi_evolve(pop_size, max_generations, max_radius, n, random_choice)

end = time.time()
print('time in seconds to complete:',end - start)
print('The best radius is:', max(best_of_each_gen))
print('The best radius with random choice is:', max(rand_best_of_each_gen))
#The best radius is: (0.02254695027135184, 0.11764705882352941)
#The best radius with random choice is: (0.02254695027135184, 0.11764705882352941)
#with pop_size=100 we obtain the same solution with both!!!!

#TESTS:
#only decreasing the pop_size you can actally see a difference
#if you only modify the max_generations you get the same result with both methods
#wheel selection might not be the best choice in this case because all the radiuses have pretty similar fitness values



Evolutionary period begins


100%|██████████| 1000/1000 [00:01<00:00, 631.70it/s]

Evolutionary period ended
time in seconds to complete: 1.5932748317718506
The best radius is: (0.01870081198259601, 1.6470588235294117)
The best radius with random choice is: (0.01870081198259601, 1.6470588235294117)


In [85]:
#Testing with different n
bits=range(7,15,1)
pop_size=10
max_generation=100
max_radius=30
results=[]
for k in bits:
    best_of_each_gen = digi_evolve(pop_size, max_generations, max_radius, k, wheel_selection)
    results.append(max(best_of_each_gen))
print(results)

Evolutionary period begins


100%|██████████| 1000/1000 [00:01<00:00, 744.99it/s]


Evolutionary period ended
Evolutionary period begins


100%|██████████| 1000/1000 [00:01<00:00, 698.40it/s]


Evolutionary period ended
Evolutionary period begins


...

Evolutionary period ended
Evolutionary period begins


100%|██████████| 1000/1000 [00:01<00:00, 513.61it/s]


Evolutionary period ended
Evolutionary period begins


100%|██████████| 1000/1000 [00:02<00:00, 483.84it/s]

Evolutionary period ended
[(0.018703743102528807, 1.6535433070866141), (0.0176960355737777, 2.1176470588235294), (0.018479723686534884, 1.8786692759295498), (0.018414002712532847, 1.906158357771261), (0.018378496840287913, 1.9198827552515876), (0.01843564649449667, 1.8974358974358974), (0.018670631554614573, 1.6078622878769382), (0.013925164947135365, 0.899102728438015)]
